# Social Vulnerability Index (SoVI)

This notebook script is designed to process and analyze census data for Pinellas County, Florida, focusing on calculating Social Vulnerability Index (SoVI) scores at the block group level using various demographic, socioeconomic, and housing variables.

# Install Necessary packages

In [ ]:
# pip install census
# pip install us
# pip install pandas
# pip install geopandas
# pip install numpy
# pip install contextily
# pip install pyproj
# pip install matplotlib
# pip install seaborn
# pip install scikit-learn
# pip install factor_analyzer

# Load packages

An instance of the Census API is initialized with an API key to fetch data.

In [ ]:
from census import Census
from us import states
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import contextily as cx
import proj as prj

In [ ]:
c = Census("Paste your api key here")

# Defining Variables and Fetching Census Data

A set of census variables (fields) related to population, age, income, education, employment, housing, and race is defined. The script fetches data from the 2022 ACS 5-year estimates for all block groups in Pinellas County, Florida.

In [ ]:
fields = list({
    'B01001_001E', # 'Total_population',
    'B01001_026E', # 'Females',    
    'B01001_003E', # 'male Population_under_five',
    'B01001_027E', # 'female Population_under_five',
    'B01001_018E', # 'male Population_over_60_61',
    'B01001_019E', # 'male Population_over_62_64',
    'B01001_020E', # 'male Population_over_65_66',
    'B01001_021E', # 'male Population_over_67_69',
    'B01001_022E', # 'male Population_over_70_74',
    'B01001_023E', # 'male Population_over_75_79',
    'B01001_024E', # 'male Population_over_80_84',
    'B01001_025E', # 'male Population_over_85',
    'B01001_042E', # 'female Population_over_60_61',
    'B01001_043E', # 'female Population_over_62_64',
    'B01001_044E', # 'female Population_over_65_66',
    'B01001_045E', # 'female Population_over_67_69',
    'B01001_046E', # 'female Population_over_70_74',
    'B01001_047E', # 'female Population_over_75_79',
    'B01001_048E', # 'female Population_over_80_84',
    'B01001_049E', # 'female Population_over_85',  
    'B11001_001E', # 'Number_of_households',
    'B11001_006E', # 'Female_headed_households_no_spouse_present',    
    'B19001_013E', # 'Households_earning_75K_100k',
    'B19001_014E', # 'Households_earning_100k_125k',
    'B19001_015E', # 'Households_earning_125K_150k',
    'B19001_016E', # 'Households_earning_150k_200k',
    'B19001_017E', # 'Households_earning_more_than_200k',
    'B17010_002E', # 'People_living_in_poverty',
    'B25003_001E', # 'Total Housing Units'
    'B25003_003E', # 'Renter_occupied_housing_units',
    'B25024_001E', # 'Total Units in Structure'
    'B25024_010E', # 'Mobile_homes',    
    'B15003_002E', # 'No_high_school_diploma',
    'B15003_003E', # 'No_high_school_diploma',
    'B15003_004E', # 'No_high_school_diploma',
    'B15003_005E', # 'No_high_school_diploma',
    'B15003_006E', # 'No_high_school_diploma',
    'B15003_007E', # 'No_high_school_diploma',
    'B15003_008E', # 'No_high_school_diploma',
    'B15003_009E', # 'No_high_school_diploma',
    'B15003_010E', # 'No_high_school_diploma',
    'B15003_011E', # 'No_high_school_diploma',
    'B15003_012E', # 'No_high_school_diploma',   
    'B15003_013E', # 'No_high_school_diploma',    
    'B15003_014E', # 'No_high_school_diploma',    
    'B15003_015E', # 'No_high_school_diploma',
    'B15003_016E', # 'No_high_school_diploma',    
    'B23025_005E', # 'Unemployed_civilian_labor_force',        
    'B23025_002E', # 'Labor_force_participation',    
    'B01002_002E', # 'Median_age',
    'B19301_001E', # 'Per_capita_income',
    'B25077_001E', # 'Median_value_of_housing',
    'B25064_001E', # 'Median_rent',
    'B19055_001E', # 'Total Social Security recipents'
    'B19055_002E', # 'Per_capita_Social_Security_recipients',    
    'B02001_002E', # 'White',    
    'B02001_003E', # 'Black',  
    'B02001_004E', # 'American indian',  
    'B02001_005E', # 'Asian',  
    'B02001_006E', # 'Hawaiian',     
    'B03002_012E', # 'Hispanic',
    'C24030_001E', # 'Total Employment'
    'C24030_003E', # 'Male Primary_extractive_industries_employment',
    'C24030_028E', # 'Female Primary_extractive_industries_employment',
    'C24020_036E', # 'Male Transportation_employment',
    'C24020_072E', # 'Female Transportation_employment',
    'C24020_019E', # 'Male Service_occupations_employment',
    'C24020_001E', # Total occupations
    'C24020_055E', # 'Female Service_occupations_employment',
    'C24030_029E', # 'Female_labor_employment',
 
})

# FIPS codes
state_fips = "12"   # Florida
county_fips = "103" # Pinellas County

# Fetch data for block groups in Pinellas County, FL, for ACS 5-year 2022
data = c.acs5.state_county_blockgroup(
    fields=fields,
    state_fips=state_fips,
    county_fips=county_fips,
    blockgroup="*",
    year=2022 # Year of Census Data
)

In [ ]:
#Create a dataframe with all variables
data_df = pd.DataFrame(data)

In [ ]:
#Print the dataframe
data_df.head()

# Data Processing and Calculations

The script performs various calculations on the census data, such as calculating percentages for female population, children, elderly, unemployed, households earning above certain thresholds, poverty rate, rental rate, and other demographic characteristics.

In [ ]:
data_df['avghouse'] = (data_df['B11001_001E'] / data_df['B01001_001E']) * 100
data_df['female'] = (data_df['B01001_026E'] / data_df['B01001_001E']) * 100
data_df['kid'] = ((data_df['B01001_003E'] + data_df['B01001_027E']) / data_df['B01001_001E']) * 100
data_df['elderly'] = ((data_df['B01001_018E'] + data_df['B01001_019E']+ data_df['B01001_020E']+data_df['B01001_021E']+data_df['B01001_022E']+data_df['B01001_023E']+data_df['B01001_024E']+data_df['B01001_025E']+data_df['B01001_042E']+data_df['B01001_043E']+data_df['B01001_044E']+data_df['B01001_045E']+data_df['B01001_046E']+data_df['B01001_047E']+data_df['B01001_048E']+data_df['B01001_049E']) / data_df['B01001_001E']) * 100
data_df['unemployed'] = (data_df['B23025_005E'] / data_df['B23025_002E']) * 100
data_df['house75k'] = ((data_df['B19001_013E'] + data_df['B19001_014E']+ data_df['B19001_015E']+data_df['B19001_016E']+data_df['B19001_017E']) / data_df['B11001_001E']) * 100
data_df['poverty'] = (data_df['B17010_002E'] / data_df['B01001_001E']) * 100
data_df['rental'] = (data_df['B25003_003E'] / data_df['B25003_001E']) * 100
data_df['mobile'] = (data_df['B25024_010E'] / data_df['B25024_001E']) * 100
data_df['femalehous'] = (data_df['B11001_006E'] / data_df['B11001_001E']) * 100
data_df['nodiploma'] = ((data_df['B15003_002E'] + data_df['B15003_003E']+ data_df['B15003_004E']+data_df['B15003_005E']+data_df['B15003_006E'] + data_df['B15003_007E']+ data_df['B15003_008E']+data_df['B15003_009E']+data_df['B15003_010E']+data_df['B15003_011E']+data_df['B15003_012E']+data_df['B15003_013E']+data_df['B15003_014E']+data_df['B15003_015E']+data_df['B15003_016E']) / data_df['B01001_001E']) * 100
data_df['labor'] = (data_df['B23025_002E'] / data_df['B01001_001E']) * 100
data_df['femalelab'] = (data_df['C24030_029E'] / data_df['C24030_001E']) * 100
data_df['income'] = (data_df['B19301_001E'])
data_df['medhome'] = (data_df['B25077_001E'])
data_df['medage'] = (data_df['B01002_002E'])
data_df['medrent'] = (data_df['B25064_001E'])
data_df['SSrecip'] = (data_df['B19055_002E'] / data_df['B19055_001E']) * 100
data_df['white'] = (data_df['B02001_002E'] / data_df['B01001_001E']) * 100
data_df['black'] = (data_df['B02001_003E'] / data_df['B01001_001E']) * 100
data_df['amerind'] = (data_df['B02001_004E'] / data_df['B01001_001E']) * 100
data_df['asian'] = (data_df['B02001_005E'] / data_df['B01001_001E']) * 100
data_df['hawaiian'] = (data_df['B02001_006E'] / data_df['B01001_001E']) * 100
data_df['hispanic'] = (data_df['B03002_012E'] / data_df['B01001_001E']) * 100
data_df['primaryemp'] = ((data_df['C24030_003E'] + data_df['C24030_028E']) / data_df['C24030_001E']) * 100
data_df['transemp'] =  ((data_df['C24020_036E'] + data_df['C24020_072E']) / data_df['C24020_001E']) * 100
data_df['serviceemp'] =  ((data_df['C24020_019E'] + data_df['C24020_055E']) / data_df['C24020_001E']) * 100

In [ ]:
#Print dataframe with calculated variables
print(data_df.head())

# Calculating housing units per square mile

The processed data is then merged dataframe into census blockgroup shapefile, which includes calculated fields such as housing units per square mile (based on field "ALAND"). We also replace missing values as 0.

In [ ]:
# Define paths
shapefile_path = r"path to census blockgroup shapefile"
output_path = r"output path of census blockgroup shapefile with merged socioeconomic data"

# Load shapefile
gdf = gpd.read_file(shapefile_path)

# Ensure GEOID is string
gdf['GEOID'] = gdf['GEOID'].astype(str)
data_df['GEOID'] = data_df['GEOID'].astype(str)

# Filter GEOIDs for Pinellas County
gdf_filtered = gdf[gdf['GEOID'].str.startswith('12103')]
data_df_filtered = data_df[data_df['GEOID'].str.startswith('12103')]

# Calculate square miles from ALAND (in sq meters)
SQM_TO_SQMILE = 3.861e-7
gdf_filtered['SQMILES'] = gdf_filtered['ALAND'] * SQM_TO_SQMILE

# Merge shapefile with ACS data
merged_filtered_df = gdf_filtered.merge(data_df_filtered, on='GEOID', how='left')

# Calculate housing units per square mile
merged_filtered_df['housesqml'] = merged_filtered_df['B25003_001E'] / merged_filtered_df['SQMILES']

# Select relevant columns for export
columns = ['GEOID','avghouse','female','kid','elderly','unemployed','house75k','poverty','rental','mobile',
           'femalehous','nodiploma','labor','femalelab','medage','income','medhome','medrent','SSrecip',
           'white','black','amerind','asian','hawaiian','hispanic','primaryemp','transemp','serviceemp',
           'housesqml']

# Ensure all selected columns exist
available_columns = [col for col in columns if col in merged_filtered_df.columns]

# Add geometry for shapefile export
final_gdf = merged_filtered_df[available_columns + ['geometry']]

#replacing missing values with NaN
final_gdf.replace(-666666666.0, np.nan, inplace=True)

In [ ]:
#Print final geodataframe
final_gdf.head()

# Adjusting missing values using average of neighboring polygons

A function is implemented to fill in missing values in the dataset by averaging values from neighboring polygons. The GeoDataFrame with filled missing values is saved as a new shapefile. This is developed by Alec Colarusso (PhD. Student and  Web Interface & App Development Lead, University of South Florida) 

In [ ]:
# Identify columns with missing values (represented as 0)
missing_cols = [col for col in final_gdf.columns if (final_gdf[col] == 0).any()]

def fill_missing_values(final_gdf, missing_cols, buffer_distance=0.1, max_iterations=5): #adjust buffer distance based on user requirements
    # Convert columns to float
    for col in missing_cols:
        final_gdf[col] = final_gdf[col].astype(float)
    
    iteration = 0
    while iteration < max_iterations:
        final_gdf['geometry'] = final_gdf['geometry'].buffer(0)  # Clean geometry
        
        for idx, row in final_gdf.iterrows():
            # Expand the current block to find neighbors
            search_area = row.geometry.buffer(buffer_distance)
            neighbors = final_gdf[final_gdf.geometry.intersects(search_area)].index

            for col in missing_cols:
                if final_gdf.at[idx, col] == 0:  # Treat 0 as missing value
                    # Calculate the mean of the neighbors' values for the missing column
                    neighbor_values = final_gdf.loc[neighbors, col]
                    neighbor_values = neighbor_values[neighbor_values != 0]  # Exclude other missing values
                    if not neighbor_values.empty:
                        mean_value = neighbor_values.mean()
                        final_gdf.at[idx, col] = mean_value
        
        # Check if there are still any 0 values
        remaining_zeros = final_gdf[missing_cols].eq(0).any(axis=1)
        if not remaining_zeros.any():
            break
        
        iteration += 1
    
    return final_gdf

# Apply the function to fill missing values
final_gdf_filled = fill_missing_values(final_gdf, missing_cols)

# Save the updated shapefile
output_path = 'Output path of shapefile after adjusted missing values'
final_gdf_filled.to_file(output_path)

# SoVI Calculation

We adopted the methodology by Cutter et al. (2003) to calculate the Social Vulnerability Index (SoVI) in block groups. All variables were normalized through z-score standardization. Using principal component analysis, the 28 socioeconomic variables were reduced into a few components to represent different dimensions of social vulnerability. The cardinality (+ or – sign) of the components is determined by the prior knowledge on the tendency of the phenomena to increase or decrease vulnerability. Finally, all the components with their cardinality (+,-) are placed in an additive model to generate the overall SoVI® score. Details of this method can be found in Cutter et al. (2003).

Reference: Cutter, S.L.; Boruff, B.J.; Shirley, W.L. Social vulnerability to environmental hazards. Soc .Sci. Q. 2003, 84, 242–261. https://www.jstor.org/stable/42955868.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer

# --------------------- Configuration --------------------- #
shapefile_path = r'output path of census blockgroup shapefile with adjusted missing values'
loadings_output_path = r'output path of detail breakdown of factor loadings output as csv'
variance_output_path = r'output path of detail breakdown of variance explained by each factor loadings output as csv'
output_shapefile_path = r'output path of census blockgroup shapefile with SoVI score'

# --------------------- Variables --------------------- #
variables = [
    'avghouse', 'female', 'kid', 'elderly', 'unemployed', 'house75k', 'poverty', 'rental',
    'mobile', 'femalehous', 'nodiploma', 'labor', 'femalelab', 'medage', 'income',
    'medhome', 'medrent', 'SSrecip', 'white', 'black', 'amerind', 'asian',
    'hawaiian', 'hispanic', 'primaryemp', 'transemp', 'serviceemp', 'housesqml'
]

# --------------------- Load and Preprocess --------------------- #
gdf = gpd.read_file(shapefile_path)
data = gdf[variables].copy()
data_imputed = SimpleImputer(strategy='mean').fit_transform(data)
data_scaled = StandardScaler().fit_transform(data_imputed)

# --------------------- PCA for Scree Plot --------------------- #
pca = PCA()
pca.fit(data_scaled)
eigenvalues = pca.explained_variance_
explained_var = pca.explained_variance_ratio_ * 100
components = range(1, len(eigenvalues) + 1)

# Plot Scree
sns.set_style("darkgrid")
fig, ax1 = plt.subplots(figsize=(8, 5))
ax1.plot(components, explained_var, marker='o', color='royalblue')
ax1.set_xlabel('Principal Component', color='black')
ax1.set_ylabel('% of variance explained', color='black')

ax2 = ax1.twinx()
ax2.plot(components, eigenvalues, marker='o', linestyle='-', color='royalblue')
ax2.set_ylabel('Eigenvalue', color='black')
ax2.axhline(y=1, color='red', linestyle='--')

for spine in ax1.spines.values(): spine.set_color('navy')
for spine in ax2.spines.values(): spine.set_color('navy')

plt.tight_layout()
plt.savefig(scree_output_path, format='svg', dpi=300)
plt.close()

# --------------------- Factor Analysis --------------------- #
fa_temp = FactorAnalyzer(rotation=None, method='principal')
fa_temp.fit(data_scaled)
eigenvals, _ = fa_temp.get_eigenvalues()
n_factors = 9  # Based on eigenvalue > 1

fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax', method='principal')
fa.fit(data_scaled)

# --------------------- Factor Loadings & Variance --------------------- #
loadings = pd.DataFrame(fa.loadings_, index=variables, columns=[f'Factor{i}' for i in range(1, n_factors + 1)])
loadings.to_csv(loadings_output_path)

variance_info = pd.DataFrame({
    "Factor": [f"Factor{i}" for i in range(1, n_factors + 1)],
    "Variance Explained (%)": fa.get_factor_variance()[1],
    "Cumulative Variance (%)": fa.get_factor_variance()[2]
})
variance_info.to_csv(variance_output_path, index=False)

# --------------------- Compute Factor Scores --------------------- #
factor_scores = fa.transform(data_scaled)
factor_scores_df = pd.DataFrame(factor_scores, columns=[f'Factor{i}' for i in range(1, n_factors + 1)])
factor_scores_df = (factor_scores_df - factor_scores_df.mean()) / factor_scores_df.std()

# --------------------- Apply Directional Signs --------------------- #
# Signs derived from interpretation
signs = {
    'Factor1': '+',  # Elderly Age Dependency
    'Factor2': '-',  # Affluence and Housing Investment
    'Factor3': '+',  # Racial and Economic Marginalization
    'Factor4': '+',  # Gender-Based Structural Inequality
    'Factor5': '+',  # Urban Density and Rental Exposure
    'Factor6': '+',  # Educational Deficits and Minority Status
    'Factor7': '+',  # Native American and Hispanic Presence
    'Factor8': '+',  # Pacific Islander Concentration
    'Factor9': '+'   # Child Dependents and Precarious Labor
}

for factor, sign in signs.items():
    if sign == '-':
        factor_scores_df[factor] *= -1

# --------------------- Compute Final SoVI Score --------------------- #
gdf['SoVI'] = factor_scores_df.sum(axis=1)

# --------------------- Classification --------------------- #
# Quantile-based
gdf['SoVI_Q'] = pd.qcut(gdf['SoVI'], q=5, labels=False)
gdf['SoVI_QS'] = pd.qcut(gdf['SoVI'], q=5, labels=['Very Low', 'Low', 'Moderate', 'High', 'Very High'])

# SD-based
mean_sovi = gdf['SoVI'].mean()
std_sovi = gdf['SoVI'].std()
bins_sd = [
    gdf['SoVI'].min(),
    mean_sovi - 1.5 * std_sovi,
    mean_sovi - 0.5 * std_sovi,
    mean_sovi + 0.5 * std_sovi,
    mean_sovi + 1.5 * std_sovi,
    gdf['SoVI'].max()
]
gdf['SoVI_SD'] = gdf['SoVI']
gdf['SoVI_SDS'] = pd.cut(gdf['SoVI'], bins=bins_sd, labels=['Very Low', 'Low', 'Moderate', 'High', 'Very High'], include_lowest=True)

# Convert labels to string to ensure compatibility with shapefile
gdf['SoVI_QS'] = gdf['SoVI_QS'].astype(str)
gdf['SoVI_SDS'] = gdf['SoVI_SDS'].astype(str)

# --------------------- Export Final Shapefile --------------------- #
gdf.to_file(output_shapefile_path)